## **Color Theory Analysis using AI**
Color theory is a crucial aspect of design, affecting how we interpret aesthetics, convey messages, and evoke emotions. It encompasses the study of color relationships and the effects of color combinations in visual compositions. Traditional color theory analysis involves understanding color wheels, primary and secondary colors, complementary colors, and the impact of different hues and saturations on viewer perception.

Artificial Intelligence (AI) has the potential to significantly ease and enhance the process of color theory analysis. AI can automate and optimize color selection, helping designers by suggesting harmonious color palettes based on desired emotions or themes. Additionally, AI-driven tools can analyze large datasets of images and designs to identify trending colors and patterns, providing insights that are not immediately apparent to the human eye.

AI can also facilitate personalized color recommendations, adapting to individual preferences or specific brand guidelines. This customization extends to accessibility considerations, where AI can ensure that color palettes are perceivable by people with color vision deficiencies, enhancing the universal usability of designs.



File upload

In [1]:

from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving shape_predictor_68_face_landmarks.dat to shape_predictor_68_face_landmarks.dat
User uploaded file "shape_predictor_68_face_landmarks.dat" with length 99693937 bytes


Libraries to install

In [2]:
pip install opencv-python dlib numpy openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


I have divided the code in two parts first, to analyse the face and get face colors and secondly integrating it with open ai and running it together to generate analysis report

In [1]:
#importing libraries
import cv2
import dlib
import numpy as np
from google.colab.patches import cv2_imshow

# Load the face detector and shape predictor models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#function  to get the color code by averaging pixel values.
def get_average_color(image, region):
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, [region], (255, 255, 255))
    mean_color = cv2.mean(image, mask=mask[:, :, 0])
    return mean_color[:3]



RuntimeError: Unable to open shape_predictor_68_face_landmarks.dat

The following code maps the face and calculate the the average of skin color and gives us the result in BGR Value.
If you are using this code on colab, please make sure you use cv2_imshow else you can use cv2.imshow() as well.

In [2]:
# Load the image
image = cv2.imread("image.jpeg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = detector(gray)
for face in faces:
    landmarks = predictor(gray, face)

    # Extract regions based on landmarks
    forehead = np.array([(landmarks.part(19).x, landmarks.part(19).y),
                         (landmarks.part(24).x, landmarks.part(24).y),
                         (landmarks.part(24).x, landmarks.part(24).y - 50),
                         (landmarks.part(19).x, landmarks.part(19).y - 50)])

    nose = np.array([(landmarks.part(27).x, landmarks.part(27).y),
                     (landmarks.part(33).x, landmarks.part(33).y),
                     (landmarks.part(31).x, landmarks.part(31).y),
                     (landmarks.part(35).x, landmarks.part(35).y)])

    left_cheek = np.array([(landmarks.part(1).x, landmarks.part(1).y),
                           (landmarks.part(3).x, landmarks.part(3).y),
                           (landmarks.part(31).x, landmarks.part(31).y),
                           (landmarks.part(48).x, landmarks.part(48).y),
                           (landmarks.part(36).x, landmarks.part(36).y)])

    right_cheek = np.array([(landmarks.part(15).x, landmarks.part(15).y),
                            (landmarks.part(13).x, landmarks.part(13).y),
                            (landmarks.part(35).x, landmarks.part(35).y),
                            (landmarks.part(54).x, landmarks.part(54).y),
                            (landmarks.part(45).x, landmarks.part(45).y)])

    lips = np.array([(landmarks.part(48).x, landmarks.part(48).y),
                     (landmarks.part(49).x, landmarks.part(49).y),
                     (landmarks.part(50).x, landmarks.part(50).y),
                     (landmarks.part(51).x, landmarks.part(51).y),
                     (landmarks.part(52).x, landmarks.part(52).y),
                     (landmarks.part(53).x, landmarks.part(53).y),
                     (landmarks.part(54).x, landmarks.part(54).y),
                     (landmarks.part(55).x, landmarks.part(55).y),
                     (landmarks.part(56).x, landmarks.part(56).y),
                     (landmarks.part(57).x, landmarks.part(57).y),
                     (landmarks.part(58).x, landmarks.part(58).y),
                     (landmarks.part(59).x, landmarks.part(59).y)])

    # Get the average color of each region
    forehead_color = get_average_color(image, forehead)
    nose_color = get_average_color(image, nose)
    left_cheek_color = get_average_color(image, left_cheek)
    right_cheek_color = get_average_color(image, right_cheek)
    lips_color = get_average_color(image, lips)

    # Print the color codes
    print("Forehead Color: ", forehead_color)
    print("Nose Color: ", nose_color)
    print("Left Cheek Color: ", left_cheek_color)
    print("Right Cheek Color: ", right_cheek_color)
    print("Lips Color: ", lips_color)

# Display the image with regions (optional)
for region in [forehead, nose, left_cheek, right_cheek, lips]:
    cv2.polylines(image, [region], isClosed=True, color=(0, 255, 0), thickness=2)

cv2_imshow(image)
cv2.waitKey(0)
cv2.destroyAllWindows()


error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


Now the task was to convert the BGR To RGB and connect it further with openAI


In [ ]:
# Convert BGR to RGB for analysis
forehead_color = forehead_color[::-1]
nose_color = nose_color[::-1]
left_cheek_color = left_cheek_color[::-1]
right_cheek_color = right_cheek_color[::-1]
lips_color = lips_color[::-1]

Now connecting with OpenAI and utilising prompt engineering to get desired results.

In [ ]:
import openai

# Set up your OpenAI API key
openai.api_key = "YOUR_OPENAI_API_KEY"

# Create the prompt with the color codes
prompt = f"""
Analyze the following skin tone color codes and provide a color theory analysis. Determine whether the skin type is autumn, spring, winter, or summer. Additionally, suggest suitable lipstick shades for the skin tone.
Forehead Color (RGB): {forehead_color}
Nose Color (RGB): {nose_color}
Left Cheek Color (RGB): {left_cheek_color}
Right Cheek Color (RGB): {right_cheek_color}
Lips Color (RGB): {lips_color}
Provide a detailed analysis.
"""

response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a color theory expert."},
            {"role": "user", "content": prompt}
        ]
    )

result=response.choices[0].message['content'].strip()

Cleaning and creating functions for the entire task and one click solution to get desired results

In [2]:
import cv2
import dlib
import numpy as np
import openai

# Set up your OpenAI API key
openai.api_key = "YOUR_OPENAI_API_KEY"

# Load the face detector and shape predictor models
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def get_average_color(image, region):
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [region], 255)
    mean_color = cv2.mean(image, mask=mask)
    return mean_color[:3]  # Return BGR

def extract_facial_colors(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = detector(gray)
    if len(faces) == 0:
        print("No faces detected.")
        return

    face = faces[0]
    landmarks = predictor(gray, face)

    # Define regions based on landmarks
    forehead = np.array([
        (landmarks.part(19).x, landmarks.part(19).y),
        (landmarks.part(24).x, landmarks.part(24).y),
        (landmarks.part(24).x, landmarks.part(24).y - 30),
        (landmarks.part(19).x, landmarks.part(19).y - 30)
    ])

    nose = np.array([
        (landmarks.part(27).x, landmarks.part(27).y),
        (landmarks.part(33).x, landmarks.part(33).y),
        (landmarks.part(31).x, landmarks.part(31).y),
        (landmarks.part(35).x, landmarks.part(35).y)
    ])

    left_cheek = np.array([
        (landmarks.part(2).x, landmarks.part(2).y),
        (landmarks.part(30).x, landmarks.part(30).y),
        (landmarks.part(28).x, landmarks.part(28).y),
        (landmarks.part(3).x, landmarks.part(3).y)
    ])

    right_cheek = np.array([
        (landmarks.part(14).x, landmarks.part(14).y),
        (landmarks.part(30).x, landmarks.part(30).y),
        (landmarks.part(28).x, landmarks.part(28).y),
        (landmarks.part(13).x, landmarks.part(13).y)
    ])

    lips = np.array([
        (landmarks.part(48).x, landmarks.part(48).y),
        (landmarks.part(54).x, landmarks.part(54).y),
        (landmarks.part(64).x, landmarks.part(64).y),
        (landmarks.part(60).x, landmarks.part(60).y)
    ])

    # Get the average color of each region
    forehead_color = get_average_color(image, forehead)
    nose_color = get_average_color(image, nose)
    left_cheek_color = get_average_color(image, left_cheek)
    right_cheek_color = get_average_color(image, right_cheek)
    lips_color = get_average_color(image, lips)

    # Convert BGR to RGB for analysis
    forehead_color = forehead_color[::-1]
    nose_color = nose_color[::-1]
    left_cheek_color = left_cheek_color[::-1]
    right_cheek_color = right_cheek_color[::-1]
    lips_color = lips_color[::-1]

    return forehead_color, nose_color, left_cheek_color, right_cheek_color, lips_color

def analyze_skin_tone_and_suggest_lipsticks(forehead_color, nose_color, left_cheek_color, right_cheek_color, lips_color):
    # Create the prompt with the color codes
    prompt = f"""
    Analyze the following skin tone color codes and provide a color theory analysis. Determine whether the skin type is autumn, spring, winter, or summer. Additionally, suggest suitable lipstick shades for the skin tone.

    Forehead Color (RGB): {forehead_color}
    Nose Color (RGB): {nose_color}
    Left Cheek Color (RGB): {left_cheek_color}
    Right Cheek Color (RGB): {right_cheek_color}
    Lips Color (RGB): {lips_color}

    Provide a detailed analysis.
    """

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a color theory expert."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content'].strip()

# Example usage
image_path = "image.jpeg"
colors = extract_facial_colors(image_path)
if colors:
    forehead_color, nose_color, left_cheek_color, right_cheek_color, lips_color = colors
    analysis = analyze_skin_tone_and_suggest_lipsticks(forehead_color, nose_color, left_cheek_color, right_cheek_color, lips_color)
    print(analysis)


AuthenticationError: Incorrect API key provided: YOUR_OPE*******_KEY. You can find your API key at https://platform.openai.com/account/api-keys.